In [ ]:
import pandas as pd
from google.colab import drive
drive.mount("/content/drive")
data=pd.read_csv('/content/drive/MyDrive/Datasets/IMDB_Dataset_SPANISH.csv')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:

import nltk
import numpy as np
import spacy
import re
import string
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.feature_extraction import text #for stop_word


#Spacy spanish piple
!python -m spacy download es_core_news_lg

2023-06-30 18:08:27.971685: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 568.0/568.0 MB 3.0 MB/s eta 0:00:00
✔ Download and installation successful
You can now load the package via spacy.load('es_core_news_lg')


In [ ]:
data.head()

,Unnamed: 0,review_en,review_es,sentiment,sentimiento
0,0,One of the other reviewers has mentioned that ...,Uno de los otros críticos ha mencionado que de...,positive,positivo
1,1,A wonderful little production. The filming tec...,Una pequeña pequeña producción.La técnica de f...,positive,positivo
2,2,I thought this was a wonderful way to spend ti...,Pensé que esta era una manera maravillosa de p...,positive,positivo
3,3,Basically there's a family where a little boy ...,"Básicamente, hay una familia donde un niño peq...",negative,negativo
4,4,"Petter Mattei's ""Love in the Time of Money"" is...","El ""amor en el tiempo"" de Petter Mattei es una...",positive,positivo


In [ ]:
print(data.value_counts('sentimiento'))
print(' ')
print('nulos')
print(data.isnull().sum())

sentimiento
negativo    25000
positivo    25000
dtype: int64
 
nulos
Unnamed: 0         0
review_en          0
sentiment          0
sentimiento        0
review_es_lemma    0
dtype: int64


In [ ]:
columns_remove = ['Unnamed: 0', 'review_en', 'sentiment']
dataset=data.drop(columns_remove ,axis=1)
data.drop_duplicates(inplace=True)


In [ ]:
def clean_text(text):
    # convert text to lowercase
    text = text.lower()

    #Remove URLs
    text=re.sub(r'http:?\S+','',text)

    # Remove punctuation characters
    text = re.sub("[%s]" % re.escape(string.punctuation), " ", text)

    # Remove non-ASCII characters, but ncluding Latin characters
    text = re.sub("([^\x00-\x7F\u00C0-\u017F])+", " ", text)


    #remove extra spaces and words with less than 2 characters
    filter = [palabra for palabra in text.split() if (len(palabra) > 2 or palabra=='no') and palabra.isalpha() ]
    text = " ".join(filter)

    return text
data['review_es_clean'] = data['review_es'].apply(lambda x: clean_text(x))


In [ ]:
from nltk.corpus import stopwords
nltk.download('stopwords')
stop=list(stopwords.words("spanish"))

def remove_stopwords(text):
    stopwords_esp = stopwords.words('spanish')
    words = text.split()
    text= [word for word in words if word not in stopwords_esp]
    return " ".join(text)
data['review_es_clean']=data['review_es_clean'].apply(lambda x: remove_stopwords(x))

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


In [ ]:
nlp = spacy.load('es_core_news_lg')

def lemmatization(text):
  doc = nlp(text)
  lemas_fila = [token.lemma_ for token in doc]
  text=' '.join(lemas_fila)
  return ' '.join(lemas_fila)
data['review_es_lemma']=data['review_es_clean'].apply(lambda x: lemmatization(x))

In [ ]:
data.head()

,Unnamed: 0,review_en,review_es,sentiment,sentimiento,review_es_clean,review_es_lemma
0,0,One of the other reviewers has mentioned that ...,Uno de los otros críticos ha mencionado que de...,positive,positivo,críticos mencionado después ver solo episodio ...,crítico mencionado después ver solo episodio e...
1,1,A wonderful little production. The filming tec...,Una pequeña pequeña producción.La técnica de f...,positive,positivo,pequeña pequeña producción técnica filmación i...,pequeño pequeño producción técnico filmación i...
2,2,I thought this was a wonderful way to spend ti...,Pensé que esta era una manera maravillosa de p...,positive,positivo,pensé manera maravillosa pasar tiempo fin sema...,pensar manera maravilloso pasar tiempo fin sem...
3,3,Basically there's a family where a little boy ...,"Básicamente, hay una familia donde un niño peq...",negative,negativo,básicamente familia niño pequeño jake piensa z...,básicamente familia niño pequeño jake pensar z...
4,4,"Petter Mattei's ""Love in the Time of Money"" is...","El ""amor en el tiempo"" de Petter Mattei es una...",positive,positivo,amor tiempo petter mattei película visualmente...,amor tiempo petter mattei película visualmente...


In [ ]:
columns_remove = ['review_es','review_es_clean']
data=data.drop(columns_remove ,axis=1)

In [ ]:
from sklearn.model_selection import train_test_split
train, test = train_test_split(data,test_size=0.30,random_state=42)
x_train,y_train=train['review_es_lemma'],train['sentimiento']
x_test,y_test=test['review_es_lemma'],test['sentimiento']
print(len(x_train),len(x_test))

35000 15000


In [ ]:
#Note1: it´s better to use bigram than unigram, but needs GPU.
#Note2: To find a min and a max, you must try several values. Recommendation max_df(0.7-0.9)
tfidf=TfidfVectorizer(ngram_range=(1,2),min_df=0.005,max_df=0.70)
X_train=tfidf.fit_transform(x_train)  #y_train no es necesario porque es la salida
X_test=tfidf.transform(x_test)  #no toca hacer fit de nuevo ya que tfidf ya hizo fit arriba en train
print("train",X_train.shape)
print("test",X_test.shape)

train (35000, 3480)
test (15000, 3480)


In [ ]:
#using kernel sigmoid and L2 regularization C=0.5
from sklearn.svm import SVC
svc=SVC(kernel='sigmoid',probability=True,verbose=True)
svc.fit(X_train,y_train)
y_pred=svc.predict(X_test)


In [ ]:
from sklearn.metrics import classification_report, confusion_matrix
print(classification_report(y_test,y_pred))
cm = confusion_matrix(y_test, y_pred)
print("matriz confusión")
print(cm)

              precision    recall  f1-score   support

    negativo       0.87      0.86      0.86      7411
    positivo       0.86      0.87      0.87      7589

    accuracy                           0.86     15000
   macro avg       0.86      0.86      0.86     15000
weighted avg       0.86      0.86      0.86     15000

matriz confusión
[[6345 1066]
 [ 980 6609]]


In [ ]:
print("Negative")
print(svc.predict(tfidf.transform(['Esta película fue una completa pérdida de tiempo. El guion era confuso y la trama carecía de coherencia. Los actores parecían aburridos y poco comprometidos con sus personajes. Definitivamente, una de las peores películas que he visto']).toarray()))
print(svc.predict(tfidf.transform(['No puedo creer que esta película haya recibido buenas críticas. La trama predecible y los diálogos cliché hicieron que fuera una experiencia aburrida. Además, los efectos especiales eran de mala calidad y la dirección carecía de originalidad' ]).toarray()))
print(svc.predict(tfidf.transform(['Una película llena de pretensiones. Intentaba ser profunda y reflexiva, pero solo logró aburrirme. Los personajes eran insulsos y no lograron despertar ninguna emoción en mí. No recomendaría perder el tiempo viendo esto' ]).toarray()))
print(svc.predict(tfidf.transform(['Esta película prometía ser una emocionante aventura, pero se quedó en un intento fallido. Los efectos especiales eran deslumbrantes, pero eso no compensó la falta de desarrollo de la trama y la falta de carisma de los personajes. No vale la pena']).toarray()))
print(svc.predict(tfidf.transform(['Una película que se autodenomina comedia, pero no hizo que ni siquiera una sonrisa se dibujara en mi rostro. Los chistes eran forzados y los actores parecían estar sobreactuando en todo momento. Evitaría esta película a toda costa' ]).toarray()))

print("Positive")
print(svc.predict(tfidf.transform(['Esta película es una verdadera joya cinematográfica. El guion es inteligente y conmovedor, manteniéndote enganchado de principio a fin. Los actores entregan interpretaciones excepcionales, y la dirección es impecable. Definitivamente, una película que no te puedes perder']).toarray()))
print(svc.predict(tfidf.transform(['Una obra maestra del cine. La historia es profundamente emotiva y te hace reflexionar sobre la vida. Los efectos visuales son impresionantes y la banda sonora es cautivadora. Los actores dan vida a personajes memorables y te sumergen por completo en la trama' ]).toarray()))
print(svc.predict(tfidf.transform(['Una película que te deja sin aliento. La acción es trepidante y las escenas de combate son coreografiadas de manera brillante. Además, el guion inteligente y lleno de giros inesperados te mantiene en vilo hasta el último minuto. Definitivamente, una experiencia cinematográfica inolvidable' ]).toarray()))
print(svc.predict(tfidf.transform(['Una comedia ingeniosa y refrescante que te hará reír a carcajadas. Los diálogos son hilarantes y los actores tienen una química perfecta. Además, la dirección logra capturar la esencia de la historia de una manera divertida y entretenida. Una película que te dejará de buen humor']).toarray()))
print(svc.predict(tfidf.transform(['Una película que te transporta a un mundo de fantasía asombroso. Los efectos especiales son impresionantes y te sumergen por completo en ese universo mágico. La historia es conmovedora y los personajes te roban el corazón. Una película que te hace creer en la magia del cine' ]).toarray()))




Negative
['negativo']
['negativo']
['negativo']
['negativo']
['negativo']
Positive
['positivo']
['positivo']
['positivo']
['positivo']
['positivo']


In [ ]:
import pickle
with open('tfidf_movie_svm.pkl', 'wb') as file:
    pickle.dump(tfidf, file)
with open('svm_movie.sav', 'wb') as file:
    pickle.dump(svc, file)

from google.colab import files
files.download('svm_movie.sav')
files.download('tfidf_movie_svm.pkl')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>